# Purpose of this notebook
In order to display data in Neuroglancer it needs to be converted to one of several formats. The one we will use is called "precomputed". TIFF is not one of the accepted formats, but there is a python package for converting TIFF data into the "precomputed" data format that Neuroglancer reads. 

This notebook covers how to convert a registered mouse brain TIFF volume to precomputed format so that you can load it into Neuroglancer. It also goes through how to load your registered data and the atlas in Neuroglancer so you can see them overlaid.  


## A quick note about Neuroglancer
Neuroglancer loads in datasets in "layers". A layer can be of type "image" (like what you would get as output from the light sheet microscope) or type "segmentation" (like the atlas annotation volume). The naming is a little confusing because both layer types refer to volumes (3-d objects). 

# Setup
In order to run the code in this notebook, you will need an anaconda installation (or some other virtual environment manager) that is able to create environments running python3 and containing some additional libraries. 

In this example, I will create a virtual environment using anaconda called "ng_lightsheet".

In the terminal:
- conda create -n ng_lightsheet python=3.7.4 -y # creates the anaconda environment
- conda activate ng_lightsheet # (or "source activate ng_lightsheet", depending on which version of conda you have)
- pip install neuroglancer --use-feature=2020-resolver
- pip install cloud-volume --use-feature=2020-resolver
- pip install SimpleITK --use-feature=2020-resolver


\# To enable you to use jupyter notebooks to work with this environment as a kernel:
- pip install --user ipykernel --use-feature=2020-resolver
- python -m ipykernel install --user --name=ng_lightsheet

Once this is all installed, make sure to select this conda environment as the kernel when running this notebook (you might have to restart jupyter)

In [7]:
import os
import numpy as np
from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
import SimpleITK as sitk

from concurrent.futures import ProcessPoolExecutor

import neuroglancer


# Point to the registered data volume file. This is the output
# from running elastix to register your data to the Paxinos atlas
dataset = '20190510_fiber_placement'
animal_id = 'm360_dorsal_up'
reg_vol_path = os.path.join(
    '/jukebox/wang/willmore/lightsheet',
    dataset,
    animal_id,
    'paxinos_registration/elastix/result.1.tif'
)

# Decide on a folder name where your layer for this volume is going to live. 
basedir = '/home/ahoag/progs/fiber_tracking_example' # change to somewhere on your filesystem or bucket

layer_dir = os.path.join(basedir,dataset,f'registered_data_{animal_id}')
# Make the layer directory (mkdir won't overwrite)
mkdir(layer_dir)
print(f"created {layer_dir}")
    
# Finally, decide how many cpus you are willing and able to use for the parallelized conversion (see step 2)
cpus_to_use = 8 # 6 or more should be perfectly fast for the task 

created /home/ahoag/progs/fiber_tracking_example/20190510_fiber_placement/registered_data_m360_dorsal_up


# Step 1: Write the instructions ("info") file that will tell Neuroglancer about your data volume 
This info file is a JSON file (looks like a python dictionary but saved in a readable file) and it contains general things about our layer like the shape and physical resolution of the volume. 

In [ ]:
def make_info_file(resolution_xyz,volume_size_xyz,layer_dir):
    """ Make an JSON-formatted file called the "info" file
    for use with the precomputed data format that Neuroglancer can read.  
    --- parameters ---
    resolution_xyz:      A tuple representing the size of the pixels (dx,dy,dz) 
                         in nanometers, e.g. (20000,20000,5000) for 20 micron x 20 micron x 5 micron
    
    volume_size_xyz:     A tuple representing the number of pixels in each dimension (Nx,Ny,Nz)

                         
    layer_dir:           The directory where the precomputed data will be
                         saved
    """
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'image', # 'image' here since this layer represents imaging data
        data_type = 'uint16', # 32-bit not necessary for data usually. Use smallest possible  
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = resolution_xyz, # X,Y,Z values in nanometers, 40 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1,1024 ], # rechunk of image X,Y,Z in voxels. We want fast access to y (coronal) planes
        volume_size = volume_size_xyz, # X,Y,Z size in voxels
    )

    vol = CloudVolume(f'file://{layer_dir}', info=info)
    vol.provenance.description = "A test info file" # can change this if you want a description
    vol.provenance.owners = [''] # list of contact email addresses
    # Saves the info and provenance files for the first time
    vol.commit_info() # generates file://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates file://bucket/dataset/layer/provenance json file
    print("Created CloudVolume info file: ",vol.info_cloudpath)

    return vol

In [ ]:
## Make the info file 

# The resolution of our registered data volume is governed
# by the resolution of the Paxinos atlas, which is 10x100x10 micron resolution,
# but we need to specify it in nanometers here
resolution_xyz = (10000,100000,10000) # 10x100x10 micron resolution)
# Load the registered data volume and get its shape. 
# This can take a ~10-20 seconds to load
reg_vol = np.array(sitk.GetArrayFromImage(
    sitk.ReadImage(reg_vol_path)),dtype=np.uint16,order='F')
z_dim,y_dim,x_dim = reg_vol.shape
volume_size_xyz = (x_dim,y_dim,z_dim)

# Write the info file
vol = make_info_file(
    resolution_xyz=resolution_xyz,
    volume_size_xyz=volume_size_xyz,
    layer_dir=layer_dir)

# Step 2: Convert volume to precomputed data format
First we create a directory (the "progress_dir") at the same folder level as the layer directory to keep track of the progress of the conversion. 
All the conversion does is copy the numpy array representing the 3d volume to a new object "vol". This is done one plane at a time (although it is parallelized). As each plane is converted, an empty file is created in the progress_dir with the name of the plane. By the end of the conversion, there should be as many files in this progress_dir as there are z planes. 

In [2]:
layer_name = layer_dir.split('/')[-1]
parent_dir = '/'.join(layer_dir.split('/')[:-1])
progress_dir = mkdir(parent_dir+ f'/progress_{layer_name}') # unlike os.mkdir doesn't crash on prexisting 
print(f"created directory: {progress_dir}")

created directory: /home/ahoag/progs/fiber_tracking_example/20190510_fiber_placement/progress_registered_data_m360_dorsal_up


In [ ]:
def process_slice(y):
    """ This function copies a 2d image slice from the atlas volume
    to the cloudvolume object, vol. We will run this in parallel over 
    all y (coronal) planes
    ---parameters---
    y:    An integer representing the 0-indexed coronal slice to be converted
    """
    if os.path.exists(os.path.join(progress_dir, str(y))):
        print(f"Slice {y} already processed, skipping ")
        return
    if y >= y_dim: # y is zero indexed and runs from 0-(y_dim-1)
        print("Index {y} >= y_dim of volume, skipping")
        return
    print('Processing slice y=',y)
    array = reg_vol[:,y,:].reshape((z_dim,1,x_dim)).T
    vol[:,y,:] = array
    touch(os.path.join(progress_dir, str(y)))
    return "success"




In [ ]:
# Run the conversion in parallel. It's not a huge amount of processing but the more cores the better

# First we check to see if there are any planes that have already been converted 
# by checking the progress dir
done_files = set([ int(y) for y in os.listdir(progress_dir) ])
all_files = set(range(vol.bounds.minpt.y, vol.bounds.maxpt.y)) 

to_upload = [ int(y) for y in list(all_files.difference(done_files)) ]
to_upload.sort()
print("Remaining slices to upload are:",to_upload)
with ProcessPoolExecutor(max_workers=cpus_to_use) as executor:
    for result in executor.map(process_slice,to_upload):
        try:
            print(result)
        except Exception as exc:
            print(f'generated an exception: {exc}')

# Step 3: Host the precomputed data on your machine so that Neuroglancer can see it
This step is really easy! Note: Exectuing the code below will cause your jupyter notebook to hang, so it is better to run the following code in a new ipython terminal or a script (make sure to have the ng_lightsheet conda environment activated in that python session) rather than this notebook. 

```python
from cloudvolume import CloudVolume
vol = CloudVolume(f'file://{layer_dir}') # need to set layer_dir to the layer_dir you defined above
vol.viewer(port=1338)
```

# Step 4: View your registered data and the Paxinos atlas in Neuroglancer
Step 3 hosts your data via http on port 1338 of your local machine. To actually view your data in Neuroglancer, there are two ways to do this. You can either load the data in manually in the browser or load it in with Python. The Python method is a lot more convenient and gives you a lot more power. However, it is useful to know how to manually add a layer for debugging the Python method. 

For the manual method, open up the Braincogs Neuroglancer client: [https://nglancer.pni.princeton.edu](https://nglancer.pni.princeton.edu) (you must be using a Princeton VPN or on campus) in your browser of choice (Chrome preferred) and then click the "+" in the upper left hand corner of the screen once the black screen loads. To load in your data, type the following into the source text box:<br>
> precomputed://http://localhost:1338 <br>

Then hit tab and name your layer if you'd like. Hit enter or the "add layer" button and your layer should load into Neuroglancer. Hopefully the labels you added should be showing up in the bottom left when you hover over a region. 

For the Python method, you can do this by executing the following cells. Make sure you have hosted the data in another Python instance somewhere on your local machine at port 1338, as described in step 3.

In [3]:
# Set which client you want to use - use the BRAINCOGS client to get the latest features.
# Need to be in the Princeton VPN or on campus to use this
neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')

In [8]:
# Make a viewer object that represents your connection to a Neuroglancer session
viewer = neuroglancer.Viewer()

# Load in the layer we just made and the Paxinos mouse atlas 
# and the layer showing the Paxinos boundaries

# This cell generates a link, which when clicked brings you to the
# Neuroglancer browser interface with your data loaded in as one layer, the
# Paxinos atlas loaded in as a second layer (colored regions),
# and the Paxinos atlas boundaries as a third layer
with viewer.txn() as s:
    s.layers[layer_name] = neuroglancer.ImageLayer(
        source='precomputed://http://localhost:1338')
    s.layers['Paxinos Mouse Atlas'] = neuroglancer.SegmentationLayer(
        source='precomputed://gs://wanglab-pma/kimatlas')
    s.layers['Paxinos Boundaries'] = neuroglancer.SegmentationLayer(
        source='precomputed://gs://wanglab-pma/kimatlas_boundaries')

print(viewer)

http://127.0.0.1:46840/v/d3a194b2ec12d71b3deafc9028bf385211537b52/


After clicking the link, if all went well, you should see your data, the Paxinos colored regions and the boundaries in sagittal, coronal and horizontal views.

The coronal sections are all we care about for Paxinos (since the atlas is too low resolution to be useful in the other cross sectional views), and we also only care about the boundaries. The following code changes the layout so we only see the coronal view and rotates the brain so it is in the usual orientation. 

In [9]:
with viewer.txn() as s:
    s.layout = 'xz'
    s.crossSectionOrientation = [
    0,
    0.7071067690849304,
    0,
    0.7071067690849304
  ]

Next, we want to turn down the transparency of the colored regions so we can only see the boundaries:

In [10]:
with viewer.txn() as s:
    atlas_layer = s.layers['Paxinos Mouse Atlas']
    atlas_layer.selected_alpha = 0.0

Even though we turned the transparency of this layer all the way down, when we hover over the middle area of a region, we can still see the region name in the layer box at the top of the viewer. If you don't care about this feature you can remove the "Paxinos Mouse Atlas" layer entirely and just work only with the boundaries. The name of the region will still appear when you hover over a boundary, but because they are so thin it can be more convenient to keep this other hidden layer on.  

You will notice that the boundaries are still colored. Assuming you want to make these all black or white, you first should check the "Set all segments black/white" checkbox in Neuroglancer under the "Render" tab of the "Paxinos Boundaries" side panel. This will by defaul make the segments white. If you want them to be black, you can use the "Saturation" slidebar manually and set it to 0 or you can do that with the following code: 

In [11]:
with viewer.txn() as s:
    boundaries_layer = s.layers['Paxinos Boundaries']
    boundaries_layer.saturation = 0.0

The transparency of the boundaries can also be controlled via the Opacity (on) slider or via this example code:

In [12]:
with viewer.txn() as s:
    boundaries_layer = s.layers['Paxinos Boundaries']
    boundaries_layer.selected_alpha = 0.9

The colormap of your data can be adjusted via the "d" and "f" keys and the colors can be inverted using the "i" key. This is especially helpful for getting a better contrast against black boundaries. These can also be controlled more precisely via code, e.g.

In [ ]:
with viewer.txn() as s:
    boundaries_layer = s.layers['Paxinos Boundaries']
    boundaries_layer.shader = "void main() {emitGrayscale(1.0-toNormalized(getDataValue())*40.0);}"
    # The "1.0-" part inverts the colormap and the factor of 40.0 at the end controls the brightness. 
    # Feel free to change the factor to suit whatever works 